In [25]:
!git clone "https://github.com/FinanceData/marcap.git" marcap

fatal: destination path 'marcap' already exists and is not an empty directory.


In [81]:
def today_mc(symbol):
    """파라미터로 넘겨준 기업 코드의 오늘 기준 시가총액 리턴"""
    
    df = pd.read_csv('marcap/data/marcap-2021.csv.gz', dtype={'Code':str}, parse_dates=['Date'])
    mc = list(df[df['Code'] == symbol]['Marcap'])
    if mc is not None:
        return mc[-1]
    else:
        return None

today_mc('005930')

489522169100000.0

In [51]:
def to_float(str):
    """쉼표가 포함된 문자열을 float타입으로 변형"""
    
    result = ''
    num_list = str.split(',')
    for num in num_list:
        result += num
    return float(result)

In [57]:
import FinanceDataReader as fdr
import OpenDartReader
import pandas as pd 
import numpy as np
import datetime

pd.set_option('display.max.columns', 50)

api_key = '6ba1ec27f6281e7470d20ddef6b0000ddb901e31'
dart = OpenDartReader(api_key)

krx_df = fdr.StockListing('KRX')
symbols = krx_df['Symbol']

date= datetime.date(2019, 4, 1)
today = date.today()

information = pd.DataFrame(columns = ['Corporation', 'PER', 'PBR', 'ROE', 'Debt_R', 'Operating_R'])

In [83]:
for symbol in symbols[:10]:
    """모든 상장 기업들의 코드를 하나씩 돌며 information 데이터프레임에 저장"""
    
    true_year = today.year   # 사업보고서를 추출한 년도
    try:
        """금년->작년->재작년 사업보고서 순으로 탐색"""
        chart = dart.finstate(symbol, true_year, reprt_code='11011')
        if chart is None:
            chart = dart.finstate(symbol, true_year-1, reprt_code='11011')
            true_year -= 1
        if chart is None:
            chart = dart.finstate(symbol, true_year-1, reprt_code='11011')
            true_year -= 1
        if chart is None:
            continue
        
    except:
        """기업코드 데이터가 없음"""
        continue
    prev_chart = dart.finstate(symbol, true_year-1, reprt_code='11011')
    
    corp = dart.company(symbol)['stock_name']   # 주식 이름
    mc = today_mc(symbol)   # 시가총액
    pure = to_float(chart.loc[12, 'thstrm_amount'])   # 당기순이익
    asset =  to_float(chart.loc[2, 'thstrm_amount'])   # 총 자산
    equity = to_float(chart.loc[8, 'thstrm_amount'])   # 총 자본   
    debt = to_float(chart.loc[5, 'thstrm_amount'])   # 총 부채
    curr_operating =  to_float(chart.loc[10, 'thstrm_amount'])  # 당기 영업이익
    prev_operating = to_float(prev_chart.loc[10, 'thstrm_amount'])   # 전기 영업이익
    
    per = round(mc/pure, 2)
    pbr = round(mc/(asset-debt), 2)
    roe = round(100*pure/equity, 2)
    debt_r = round(100*debt/equity, 2)
    operating_r = round(100*(curr_operating-prev_operating)/prev_operating, 2)
    
    temp_list = [(corp, per, pbr, roe, debt_r, operating_r)]
    temp = pd.DataFrame(temp_list, columns = ['Corporation', 'PER', 'PBR', 'ROE', 'Debt_R', 'Operating_R'])
    information = information.append(temp, ignore_index=True)
    
information    

,Corporation,PER,PBR,ROE,Debt_R,Operating_R
0,AJ네트웍스,4.99,0.38,7.58,423.61,-127.56
1,AK홀딩스,1.84,0.26,14.02,130.54,1.53
2,APS홀딩스,132.15,0.65,0.50,20.21,-1509.72
3,AP시스템,16.33,3.87,23.71,295.61,74.90
4,AP위성,-393.02,3.21,-0.82,22.34,-67.62
...,...,...,...,...,...,...
154,AJ네트웍스,4.41,0.54,12.13,419.16,-183.85
155,AK홀딩스,6.61,0.27,4.01,202.67,-51.31
156,APS홀딩스,-33.51,0.72,-2.14,38.61,192.70
157,AP시스템,43.59,3.61,8.28,268.87,-37.87
